<a href="https://colab.research.google.com/github/rjnakano/manejo-inventarios-pdg/blob/main/Notebooks/TDG041821V6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade kmodes

In [1]:
from google.colab import drive
import pandas as pd
import numpy as np

from kmodes.kprototypes import KPrototypes

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_context('poster')
sns.set_style('white')
sns.set_color_codes()

from plotnine import *
import plotnine

ModuleNotFoundError: No module named 'google'

In [ ]:
plot_kwds = {'alpha' : 0.5, 's' : 1, 'linewidths':0}

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
# Dataset de inventarios sin dimensiones de los productos
data6 = pd.read_csv("/content/gdrive/MyDrive/TDGdata/dfNoDims.csv")

In [ ]:
data6

In [ ]:
data6['AISLE']=data6['AISLE'].astype(str)

In [ ]:
data6 =  data6.drop(['IG','HTS','PLANNING','ABC','UOM','CAT','COSTX','CONTRPERC'], axis=1)

In [ ]:
data6.sample(5)

In [ ]:
numdf = data6[list(data6.select_dtypes('float64').columns)]

In [ ]:
catdf = data6[list(data6.select_dtypes('object').columns)]

In [ ]:
from sklearn.metrics import silhouette_score
import time

dataNum=numdf.to_numpy()
dataCat=catdf.to_numpy()
N = data6.shape[0]

# gamma = 0.5 (constante)
catWeight=0.5

start_time = time.time()

# Inicializamos la matriz de distancias con ceros
scores = [ [ 0 for i in range(N) ] for j in range(N) ]

# Calculamos la mitad de la matriz distancias
for i in range(N):
  for j in range(i+1,N):
    scores[i][j]=np.sum((dataNum[i] - dataNum[j]) ** 2) + catWeight * np.sum(dataCat[i]!=dataCat[j])
print("Matriz de distancias --- %s seconds ---" % (time.time() - start_time))

# Completamos la matriz
for i in range(0,N):
  for j in range(0,i):
    scores[i][j]=scores[j][i]
print("Espejo --- %s seconds ---" % (time.time() - start_time))

In [ ]:
scores[0][0:10]

In [ ]:
catColumnsPos = [data6.columns.get_loc(col) for col in list(data6.select_dtypes('object').columns)]

In [ ]:
# gamma = 0.5 (constante)

cost = []
silueta = []
start_time = time.time()

for k in (list(range(1,6)) + [10]):
  try:
    kprototype = KPrototypes(n_jobs = -1, n_clusters = k, init = 'Huang', gamma = 0.5, random_state = 0)
    kprototype.fit_predict(data6, categorical = catColumnsPos)
    cost.append(kprototype.cost_)
    print('k: {}'.format(k))
    print("--- %s seconds ---" % (time.time() - start_time))
    if k != 1:
      silhouette_coeff = silhouette_score(scores, kprototype.labels_, metric='precomputed')
      silueta.append(silhouette_coeff)
      print("Coeficiente silueta --- %s seconds ---" % (time.time() - start_time))
      print(silhouette_coeff)

  except:
    break

In [ ]:
df_cost = pd.DataFrame({'Cluster':range(1,6), 'Cost':cost})
df_silueta = pd.DataFrame({'Cluster':range(2,6), 'Coeficiente':silueta})

In [ ]:
# Codo
plotnine.options.figure_size = (8, 4.8)
(
ggplot(data = df_cost)+
geom_line(aes(x = 'Cluster',
y = 'Cost'))+
geom_point(aes(x = 'Cluster',
y = 'Cost'))+
geom_label(aes(x = 'Cluster',
y = 'Cost',
label = 'Cluster'),
size = 10,
nudge_y = 1000) +
labs(title = 'Optimal number of cluster with Elbow Method')+
xlab('Number of Clusters k')+
ylab('Cost')+
theme_minimal()
)

In [ ]:
# Coeficiente silueta
plotnine.options.figure_size = (8, 4.8)
(
ggplot(data = df_silueta)+
geom_line(aes(x = 'Cluster',
y = 'Coeficiente'))+
geom_point(aes(x = 'Cluster',
y = 'Coeficiente'))+
geom_label(aes(x = 'Cluster',
y = 'Coeficiente',
label = 'Cluster'),
size = 10,
nudge_y = 0.1) +
labs(title = 'Optimal number of cluster with Silhouette coefficient')+
xlab('Number of Clusters k')+
ylab('Coeficiente')+
theme_minimal()
)